In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
import torch 
import numpy as np
import torch.nn as nn
import t3nsor as t3
from torchtext import data
from torchtext import datasets
import torch.optim as optim

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
TEXT = data.Field(tokenize='spacy');
LABEL = data.LabelField(dtype=torch.float);

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [6]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [7]:
train_data, valid_data = train_data.split()

In [8]:
TEXT.build_vocab(train_data, max_size=24998)
LABEL.build_vocab(train_data)

In [9]:
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE,
    device=device)

In [10]:
class LSTM_Classifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super().__init__()       
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):      
        embedded = self.dropout(x) 
        output, (hidden, cell) = self.rnn(embedded)
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        return self.fc(hidden.squeeze(0))

In [45]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 256
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5

In [46]:
lstm_model = LSTM_Classifier(embedding_dim=EMBEDDING_DIM, 
                             hidden_dim=HIDDEN_DIM, 
                             output_dim=OUTPUT_DIM,
                             n_layers=N_LAYERS,
                             bidirectional=BIDIRECTIONAL,
                             dropout=DROPOUT)

In [47]:
INPUT_DIM

25000

In [48]:
embed_model = t3.TTEmbedding(shape=[[2, 5, 2, 5, 2, 5, 5, 5], [2, 2, 2, 2, 2, 2, 2, 2]], tt_rank=16, batch_dim_last=True)

In [49]:
model = nn.Sequential(embed_model, lstm_model)

In [50]:
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

In [51]:
model = model.to(device)
criterion = criterion.to(device)

In [52]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc

In [42]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    
    for i, batch in enumerate(iterator):
        
        optimizer.zero_grad()
                

        predictions = model(batch.text).squeeze(1)
        
        
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
        if i > len(iterator):
            break
        
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [43]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):
            

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
            
            if i > len(iterator):
                break
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [44]:
N_EPOCHS = 10

for epoch in range(N_EPOCHS):

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}% |')

| Epoch: 01 | Train Loss: 0.686 | Train Acc: 56.76% | Val. Loss: 0.655 | Val. Acc: 62.19% |
| Epoch: 02 | Train Loss: 0.657 | Train Acc: 61.32% | Val. Loss: 0.616 | Val. Acc: 66.34% |
| Epoch: 03 | Train Loss: 0.596 | Train Acc: 68.90% | Val. Loss: 0.511 | Val. Acc: 75.87% |
| Epoch: 04 | Train Loss: 0.501 | Train Acc: 76.76% | Val. Loss: 0.449 | Val. Acc: 79.84% |
| Epoch: 05 | Train Loss: 0.434 | Train Acc: 80.87% | Val. Loss: 0.392 | Val. Acc: 82.98% |
| Epoch: 06 | Train Loss: 0.391 | Train Acc: 83.40% | Val. Loss: 0.378 | Val. Acc: 83.42% |
| Epoch: 07 | Train Loss: 0.387 | Train Acc: 84.08% | Val. Loss: 0.345 | Val. Acc: 85.17% |
| Epoch: 08 | Train Loss: 0.343 | Train Acc: 86.13% | Val. Loss: 0.339 | Val. Acc: 86.38% |
| Epoch: 09 | Train Loss: 0.309 | Train Acc: 87.81% | Val. Loss: 0.305 | Val. Acc: 87.52% |
| Epoch: 10 | Train Loss: 0.296 | Train Acc: 88.20% | Val. Loss: 0.300 | Val. Acc: 88.36% |


# Pretrained model

In [21]:
TEXT.build_vocab(train_data, max_size=24998, vectors="glove.6B.100d")
LABEL.build_vocab(train_data)

In [22]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE, 
    device=device)

In [23]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25000, 100])


In [24]:
pretrained_embeddings.shape

torch.Size([25000, 100])

In [25]:
tt_pretrained = t3.to_tt_matrix(pretrained_embeddings.to(device), shape=[[2, 5, 2, 5, 2, 5, 5, 5], [2, 5, 2, 5, 1, 1, 1, 1]], max_tt_rank=20)

In [26]:
print(tt_pretrained)

A TT-Matrix of size 25000 x 100, underlying tensorshape: [2, 5, 2, 5, 2, 5, 5, 5] x [2, 5, 2, 5, 1, 1, 1, 1], TT-ranks: [1, 4, 20, 20, 20, 20, 20, 5, 1] 
 on device 'cuda:0' with compression rate 147.57


In [27]:
embed_model_pretrained = t3.TTEmbedding(init=tt_pretrained)

In [28]:
model_pre = nn.Sequential(embed_model_pretrained, lstm_model)

In [29]:
optimizer = optim.Adam(model_pre.parameters())
criterion = nn.BCEWithLogitsLoss()

In [30]:
model_pre = model_pre.to(device)
criterion = criterion.to(device)

In [31]:
N_EPOCHS = 20

for epoch in range(N_EPOCHS):

    train_loss, train_acc = train(model_pre, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model_pre, valid_iterator, criterion)
    
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}% |')

| Epoch: 01 | Train Loss: 0.682 | Train Acc: 56.07% | Val. Loss: 0.609 | Val. Acc: 69.47% |
| Epoch: 02 | Train Loss: 0.614 | Train Acc: 68.10% | Val. Loss: 0.558 | Val. Acc: 71.74% |
| Epoch: 03 | Train Loss: 0.505 | Train Acc: 76.94% | Val. Loss: 0.457 | Val. Acc: 79.61% |
| Epoch: 04 | Train Loss: 0.423 | Train Acc: 81.42% | Val. Loss: 0.386 | Val. Acc: 83.03% |
| Epoch: 05 | Train Loss: 0.380 | Train Acc: 84.43% | Val. Loss: 0.372 | Val. Acc: 84.16% |
| Epoch: 06 | Train Loss: 0.358 | Train Acc: 85.54% | Val. Loss: 0.371 | Val. Acc: 83.51% |
| Epoch: 07 | Train Loss: 0.333 | Train Acc: 86.85% | Val. Loss: 0.361 | Val. Acc: 83.58% |
| Epoch: 08 | Train Loss: 0.317 | Train Acc: 87.41% | Val. Loss: 0.330 | Val. Acc: 86.13% |
| Epoch: 09 | Train Loss: 0.292 | Train Acc: 88.67% | Val. Loss: 0.336 | Val. Acc: 85.94% |
| Epoch: 10 | Train Loss: 0.284 | Train Acc: 89.04% | Val. Loss: 0.321 | Val. Acc: 86.27% |
| Epoch: 11 | Train Loss: 0.269 | Train Acc: 90.01% | Val. Loss: 0.327 | Val. Ac

KeyboardInterrupt: 